In [ ]:
import numpy as np  # 1.21.5
from typing import NamedTuple 
import matplotlib.pyplot as plt
import imagesc as imagesc
#import pandas as pd
import librosa
import librosa.display
import scipy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as backend
from tensorflow.keras import mixed_precision
from collections import deque
import time
import random
from tqdm import tqdm
import os
#from PIL import Image
#import cv2


from berechneFWHM import berechneFWHM
from calcWidthFromVector import calcWidthFromVector
from fminCallback import fminCallback
from normPhase import normPhase
from phasePulseFormer import phasePulseFormer
from plotOptimState import plotOptimState
from shgValue import shgValue
from targetFunc import targetFunc
from xcorrSpectrogram import xcorrSpectrogram

In [ ]:
### DQN-Setup

DISCOUNT = 0.9
REPLAY_MEMORY_SIZE = 50_000  # How many last steps to keep for model training
MIN_REPLAY_MEMORY_SIZE = 1_000  # Minimum number of steps in a memory to start training
MINIBATCH_SIZE = 32  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 5  # Terminal states (end of episodes)
MODEL_NAME = 'ResV2_5'
MIN_REWARD = 140  # For model save
#MEMORY_FRACTION = 0.20

# Environment settings
EPISODES = 5000
STEPS_PER_EPISODE = 350
ACTION_STEP_SIZE = 0.1
ACTION_LIMIT_POS = 14
ACTION_LIMIT_NEG = -7

mixed_precision.set_global_policy('mixed_float16')  #   Uses float16 for layer training instead of float32 for faster computing


# Exploration settings
epsilon = 1  # not a constant, going to be decayed
EPSILON_DECAY = 0.995
MIN_EPSILON = 0.001

#  Stats settings
AGGREGATE_STATS_EVERY = 1  # episodes
SHOW_PREVIEW = False



In [ ]:
### Pulse-Shaping-Setup

# Setup Values
gdd = 2.5                   # Second order dispersion
tod = -1                    # Third order dispersion 
sigma = 1                   # width of frequency function
wp = 5                      # Midpoint of frequency function
nPix = 640                  # number of pixels of pulse shaper
samplesPerPeriod = 20       # samples per period in time domain
shaperLims = [wp - 4*sigma, wp + 4*sigma]
Tp = 2*np.pi/wp
tLim = 15
tauVec = np.linspace(-7, 7, 100)
S = 4                      # number of spline support points

# Construction of frequency axis: nPix samples in shaper-interval
w0 = (shaperLims[1] - shaperLims[0]) / nPix     # frequency resolution
NN = np.ceil(2*np.pi/(Tp*w0)*samplesPerPeriod)
ww = np.arange(0,(NN))*w0     # Frequency axis
startIdx = np.argwhere(ww < shaperLims[0])
startIdx = np.reshape(startIdx,(1,len(startIdx)))
startIdx = startIdx[0][-1] + 1
ws = NN * w0        # sample rate 
Ts = 2*np.pi / ws   # sample time
T0 = 2*np.pi/w0     # duration in time domain
tt = -T0/2 + np.arange(0,(NN))*Ts     # time axis shifted so that it centers around 0

# Lambda Funktionen:
phaseFunc = lambda w: - gdd/2 * np.power((w-wp),2) + tod/6*np.power((w-wp),3)
Apos = lambda w, phaseFunc: np.exp(-(np.power((w-wp),2)) / (2*np.power(sigma,2)) + 1j * phaseFunc(w)) # positive frequency part of the spectrum

# Reference without phase distortion
AwRef = Apos(ww, lambda w: 0)
atRef = np.fft.fftshift(np.fft.ifft(AwRef))
widthRef = calcWidthFromVector(abs(atRef), tt)
shgRef = shgValue(atRef)

# Chirped pulse before pulse shaping
Aw =  Apos(ww, phaseFunc)
at = np.fft.fftshift(np.fft.ifft(Aw))
widthBefore = calcWidthFromVector(abs(at), tt)
shgBefore = shgValue(at)

wws = np.linspace(shaperLims[0], shaperLims[1], nPix)
wSupp = np.linspace(shaperLims[0], shaperLims[1], S)

# Spectrogram
ttMask = abs(tt) < tLim
timeVec1 = np.real(at[ttMask])/Ts
timeVec2 = np.power(abs(at[ttMask]),2)/Ts
SM, wVec = xcorrSpectrogram(timeVec1, timeVec2, Ts, tauVec, False)
wMask = (wVec > 0) & (wVec <  2*wp)

def extents(f):
    delta = f[1] - f[0]
    return [f[0] - delta/2, f[-1] + delta/2]

plt.imshow(abs(SM[wMask,:]), extent=extents(tauVec) + extents(wVec[wMask]))
plt.colorbar()
plt.xlabel('Time shift $\\tau $')
plt.ylabel('Frequenz $\omega$')
plt.title('Spectrogramm')

##  start spectrum for reference 

specStart = abs(SM[wMask,:])
#specStartFlat = np.reshape(specStart,(4700,))
#specStart_pre = preprocess_input(specStart)

specStart_3ch = np.expand_dims(specStart, -1)
specStart_3ch = specStart_3ch.repeat(3, axis= -1)
specStart_3ch = preprocess_input(specStart_3ch)


##
pulseFormer = lambda phaseOffsets: phasePulseFormer(Aw, phaseOffsets, startIdx)


class Config(NamedTuple):
    phaseFunc = lambda self, w: - gdd/2 * np.power((w-wp),2) + tod/6*np.power((w-wp),3)
    #pulseFormer = lambda self, phaseOffsets: phasePulseFormer(Aw, phaseOffsets, startIdx)
    Apos = lambda self, w, phaseFunc: np.exp(-(np.power((w-wp),2)) / (2*np.power(sigma,2)) + 1j * phaseFunc(w)) # positive frequency part of the spectrum

    sigma: float            # width of frequency function
    wp: float               # Midpoint of frequency function
    shaperLims: list
    nPix: int
    samplesPerPeriod: int
    atRef: list
    tLim: int
    tauVec: list
    S: int


cfg = Config(sigma, wp, shaperLims, nPix, samplesPerPeriod, atRef, tLim, tauVec, S)






In [ ]:
# For more repetitive results
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)


class PulseShaper:    
    def __init__(self):
        self.pSupp = np.random.random(S) * 2 * np.pi
        self.width = targetFunc(self.pSupp, wSupp, wws, pulseFormer, tt)

    def action(self, choice):

        if choice == 0:
            if self.pSupp[0] > ACTION_LIMIT_POS:
                self.pSupp[0] = self.pSupp[0]
            else:
                self.pSupp[0] += ACTION_STEP_SIZE

        elif choice == 1:
            if self.pSupp[0] < ACTION_LIMIT_NEG:
                self.pSupp[0] = self.pSupp[0]
            else:
                self.pSupp[0] -= ACTION_STEP_SIZE

        elif choice == 2:
            if self.pSupp[1] > ACTION_LIMIT_POS:
                self.pSupp[1] = self.pSupp[1]
            else:
                self.pSupp[1] += ACTION_STEP_SIZE

        elif choice == 3:
            if self.pSupp[1] < ACTION_LIMIT_NEG:
                self.pSupp[1] = self.pSupp[1]
            else:
                self.pSupp[1] -= ACTION_STEP_SIZE

        elif choice == 4:
            if self.pSupp[2] > ACTION_LIMIT_POS:
                self.pSupp[2] = self.pSupp[2]
            else:
                self.pSupp[2] += ACTION_STEP_SIZE

        elif choice == 5:
            if self.pSupp[2] < ACTION_LIMIT_NEG:
                self.pSupp[2] = self.pSupp[2]
            else:
                self.pSupp[2] -= ACTION_STEP_SIZE

        elif choice == 6:
            if self.pSupp[3] > ACTION_LIMIT_POS:
                self.pSupp[3] = self.pSupp[3]
            else:
                self.pSupp[3] += ACTION_STEP_SIZE

        elif choice == 7:
            if self.pSupp[3] < ACTION_LIMIT_NEG:
                self.pSupp[3] = self.pSupp[3]
            else:
                self.pSupp[3] -= ACTION_STEP_SIZE

         

        
                
        

In [ ]:
class PulseShaperEnv:
    ACTION_SPACE_SIZE = S*2
    OBSERVATION_SPACE_VALUES = (47, 100)

    def reset(self):
        self.pulseshaper = PulseShaper()
        self.episode_step = 0

        pp = scipy.interpolate.CubicSpline(wSupp, self.pulseshaper.pSupp)
        phaseOffsets = pp(wws) 
        AwShaped = pulseFormer(phaseOffsets)
        atShaped = np.fft.fftshift(np.fft.ifft(AwShaped))

        timeVec1 = np.real(atShaped[ttMask])/Ts
        timeVec2 = np.power(abs(atShaped[ttMask]),2)/Ts
        SM, wVec = xcorrSpectrogram(timeVec1, timeVec2, Ts, tauVec, False)
        wMask = (wVec > 0) & (wVec <  2*wp)

        specStart_3ch = np.expand_dims(abs(SM[wMask,:]), -1)

        specStart_3ch = specStart_3ch.repeat(3, axis= -1)
        observation = preprocess_input(specStart_3ch)

        return observation


    def step(self, action):
        self.episode_step += 1
        self.pulseshaper.action(action)

        # Hier aktuelles Spectrogram zurückgeben (Matrix)
        # Spectrogram
        pp = scipy.interpolate.CubicSpline(wSupp, self.pulseshaper.pSupp)
        phaseOffsets = pp(wws) 
        AwShaped = pulseFormer(phaseOffsets)
        atShaped = np.fft.fftshift(np.fft.ifft(AwShaped))

        timeVec1 = np.real(atShaped[ttMask])/Ts
        timeVec2 = np.power(abs(atShaped[ttMask]),2)/Ts
        SM, wVec = xcorrSpectrogram(timeVec1, timeVec2, Ts, tauVec, False)
        wMask = (wVec > 0) & (wVec <  2*wp)

        new_observation_3ch = np.expand_dims(abs(SM[wMask,:]), -1)

        new_observation_3ch = new_observation_3ch.repeat(3, axis= -1)
        new_observation = preprocess_input(new_observation_3ch)
        
        

        self.pulseshaper.width = targetFunc(self.pulseshaper.pSupp, wSupp, wws, pulseFormer, tt)

        reward = -200+( (1 / abs(self.pulseshaper.width)) * 250)

        # print('reward: {}'.format(reward))
        # print('Support Points: {}'.format(self.pulseshaper.pSupp))

        done = False
        if self.episode_step >= STEPS_PER_EPISODE or reward > MIN_REWARD:
            done = True

        return new_observation, reward, done


    def render(self):
        plotFunc = lambda x, optimValues: plotOptimState(x, optimValues, cfg, shgRef, pulseFormer)

        class optimValues(NamedTuple):
            iteration: int
            fval: float

        optimValuesPlot = optimValues(self.episode_step, self.pulseshaper.width)

        plotFunc(self.pulseshaper.pSupp, optimValuesPlot)
        
env = PulseShaperEnv()


In [ ]:


# Create models folder
if not os.path.isdir('models'):
    os.makedirs('models')

# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)
        self._log_write_dir = self.log_dir

    def set_model(self, model):
        self.model = model

        self._train_dir = os.path.join(self._log_write_dir, 'train')
        self._train_step = self.model._train_counter

        self._val_dir = os.path.join(self._log_write_dir, 'validation')
        self._val_step = self.model._test_counter

        self._should_write_train_graph = False

    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    def on_batch_end(self, batch, logs=None):
        pass

    def on_train_end(self, _):
        pass

    def update_stats(self, **stats):
        with self.writer.as_default():
            for key, value in stats.items():
                tf.summary.scalar(key, value, step = self.step)
                self.writer.flush()

class DQNAgent:
    def __init__(self):
        # Main model
        self.model = self.create_model()

        # Target network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)

        # Custom tensorboard object
        self.tensorboard = ModifiedTensorBoard(log_dir="logs/{}".format(MODEL_NAME))


        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0

    def create_model(self):

        
        conv_base = ResNet50V2(include_top= False, weights= 'imagenet', input_shape = (47,100,3))
        conv_base.trainable = False
        
        model = Sequential()

        model.add(conv_base)
        
        model.add(Flatten())  
        model.add(Dense(64, activation='relu'))

        model.add(Dense(env.ACTION_SPACE_SIZE,activation='linear',name = 'Output'))
        
        model.compile(loss="mse", optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])
        return model
    
    # Adds step's data to a memory replay array
    # (observation space, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    
    # Trains main network every step during episode
    def train(self, terminal_state, step):

        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        
        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        # Get current states from minibatch, then query NN model for Q values
        current_states = np.asarray([transition[0] for transition in minibatch]).astype('float32')  
        current_qs_list = self.model.predict(current_states)

        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array([transition[3] for transition in minibatch])  
        future_qs_list = self.target_model.predict(new_current_states)

        X = []
        y = []

        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)

        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False, callbacks=[self.tensorboard] if terminal_state else None)

        # Update target network counter every episode
        if terminal_state:
            self.target_update_counter += 1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape))[0]
        


agent = DQNAgent()

In [ ]:
ep_rewards = []
ep_widths = []
ep_width_end = []
ep_delta_width = []

# Iterate over episodes
for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):

    # Update tensorboard step every episode
    agent.tensorboard.step = episode

    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    episode_width = 0
    step = 1
    

    # Reset environment and get initial state
    current_state = env.reset()
    width_start = env.pulseshaper.width

    # Reset flag and start iterating until episode ends
    done = False
    while not done:

        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
        else:
            # Get random action
            action = np.random.randint(0, env.ACTION_SPACE_SIZE)

        new_state, reward, done = env.step(action)

        #print('reward: {}'.format(reward))

        # Transform new continous state to new discrete state and count reward
        episode_reward += reward
        episode_width += env.pulseshaper.width


        #print('episode_reward: {}'.format(episode_reward))

        # if SHOW_PREVIEW and not episode % AGGREGATE_STATS_EVERY:      ########    Render anzeige  #######
        #     env.render()

        # Every step we update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)

        current_state = new_state
        step += 1

    # Append episode reward to a list and log stats (every given number of episodes)
    print('Width End: {}'.format(env.pulseshaper.width))
    
    ep_rewards.append(episode_reward/step)
    ep_reward_end = reward
    ep_widths.append(episode_width/step)
    ep_width_end.append(env.pulseshaper.width)
    ep_delta_width.append(width_start - env.pulseshaper.width)  

    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        average_width = sum(ep_widths[-AGGREGATE_STATS_EVERY:])/len(ep_widths[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
        end_width = sum(ep_width_end[-AGGREGATE_STATS_EVERY:])/len(ep_width_end[-AGGREGATE_STATS_EVERY:])
        delta_width = sum(ep_delta_width[-AGGREGATE_STATS_EVERY:])/len(ep_width_end[-AGGREGATE_STATS_EVERY:])
        agent.tensorboard.update_stats(reward_avg=average_reward, width_avg=average_width, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon, width_end = end_width, steps = step, delta_width = delta_width)
        
        # Save model, but only when end reward is greater or equal a set value and the Agent overcame a width of 1, so it doesnt save a lucky start config
        if (ep_reward_end >= MIN_REWARD and delta_width >= 1 and epsilon == MIN_EPSILON) or episode == 5000:
            agent.model.save(f'models/{MODEL_NAME}__{end_width:_>7.2f}endWidth_{step:_>7.2f}steps_{episode}ep.model')

    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)